In [3]:
# Importante para usar llama Index en Jupyter Notebook
import nest_asyncio

nest_asyncio.apply()


In [4]:
import pandas as pd
import pickle
from llama_index.core import VectorStoreIndex, Settings
from llama_index.core.schema import Document, NodeRelationship, RelatedNodeInfo
from GCP.lib.LlamaIndex_custom.VertexIEmbeddings import VertexIEmbeddings
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.core.extractors import QuestionsAnsweredExtractor
from llama_index.llms.vertex import Vertex



In [6]:
df = pd.read_csv(
    './data/Metadata/Bloques_y_tablas_v2-Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.csv',
    index_col=0)
df

,block_type,text,file_type,languages,page_number,filename,description,authors,research_lab,publication_year,title,section_title
4,Author Names,"Patrick Lewis'?, Ethan Perez*,",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially with a...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...
5,Author Names,"Aleksandra Piktus†, Fabio Petroni†, Vladimir K...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially affili...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...
6,Author Names,"Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, ...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,Lists the authors of the research paper.,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...
9,NarrativeText,Large pre-trained language models have been sh...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This paragraph discusses the capabilities and ...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Abstract
11,NarrativeText,Pre-trained neural language models have been s...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This block discusses the advantages and disadv...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,1 Introduction
...,...,...,...,...,...,...,...,...,...,...,...,...
216,FigureCaption,Table 7: Number of instances in the datasets u...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,"Describes the content of Table 7, indicating t...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters
217,Table,| Task | Train | Development | Test |\r\n|---|...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,Table 7: Number of instances in the datasets u...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters
218,NarrativeText,"parameters. The best performing ""closed-book"" ...",application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,This block discusses the performance of differ...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters
220,NarrativeText,"In preliminary experiments, we observed that f...",application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,This block discusses challenges encountered du...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,H Retrieval Collapse


In [7]:
# Crear el ID utilizando el índice y el nombre del archivo
df['node_id'] = df.apply(lambda row: f"{row.name}_{row['filename']}", axis=1)

# Crear la columna 'previous_node_id' desplazando el ID hacia abajo
df['previous_node_id'] = df['node_id'].shift(1)

# Crear la columna 'next_node_id' desplazando el ID hacia arriba
df['next_node_id'] = df['node_id'].shift(-1)

df

,block_type,text,file_type,languages,page_number,filename,description,authors,research_lab,publication_year,title,section_title,node_id,previous_node_id,next_node_id
4,Author Names,"Patrick Lewis'?, Ethan Perez*,",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially with a...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...,4_Retrieval-Augmented Generation for Knowledge...,None,5_Retrieval-Augmented Generation for Knowledge...
5,Author Names,"Aleksandra Piktus†, Fabio Petroni†, Vladimir K...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,"Lists the names of authors, potentially affili...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...,5_Retrieval-Augmented Generation for Knowledge...,4_Retrieval-Augmented Generation for Knowledge...,6_Retrieval-Augmented Generation for Knowledge...
6,Author Names,"Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, ...",application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,Lists the authors of the research paper.,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Retrieval-Augmented Generation for Knowledge-I...,6_Retrieval-Augmented Generation for Knowledge...,5_Retrieval-Augmented Generation for Knowledge...,9_Retrieval-Augmented Generation for Knowledge...
9,NarrativeText,Large pre-trained language models have been sh...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This paragraph discusses the capabilities and ...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,Abstract,9_Retrieval-Augmented Generation for Knowledge...,6_Retrieval-Augmented Generation for Knowledge...,11_Retrieval-Augmented Generation for Knowledg...
11,NarrativeText,Pre-trained neural language models have been s...,application/pdf,['eng'],1,Retrieval-Augmented Generation for Knowledge-I...,This block discusses the advantages and disadv...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,1 Introduction,11_Retrieval-Augmented Generation for Knowledg...,9_Retrieval-Augmented Generation for Knowledge...,12_Retrieval-Augmented Generation for Knowledg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,FigureCaption,Table 7: Number of instances in the datasets u...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,"Describes the content of Table 7, indicating t...","Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters,216_Retrieval-Augmented Generation for Knowled...,214_Retrieval-Augmented Generation for Knowled...,217_Retrieval-Augmented Generation for Knowled...
217,Table,| Task | Train | Development | Test |\r\n|---|...,application/pdf,['eng'],19,Retrieval-Augmented Generation for Knowledge-I...,Table 7: Number of instances in the datasets u...,"Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fa...","Facebook AI Research,University College London...",2020,Retrieval-Augmented Generation for Knowledge-I...,G Parameters,217_Retrieval-Augmented Generation for Knowled...,216_Retrieval-Augmented Generation for Knowled...,218_Retrieval-Augmented Generation for Knowled...
218,NarrativeText,"parameters. The best performing ""closed-book"" ...",application/pdf,['eng'],19,Retrieval-Augmented Generatio

In [8]:
def create_document(row):
    # Crear el nodo de texto con el ID, texto y metadatos
    node = Document(
        id=row['node_id'],
        text=row['text'],  # Usar la columna 'text' para el contenido del nodo
        metadata={
            'block_type': row['block_type'],
            'section_title': row['section_title'],
            'description': row['description'],
            'file_type': row['file_type'],
            'languages': row['languages'],
            'page_number': row['page_number'],
            'filename': row['filename'],
            'authors': row['authors'],
            'research_lab': row['research_lab'],
            'publication_year': row['publication_year'],
            'title_of_the_document': row['title']
        },
        excluded_llm_metadata_keys=['authors', 'research_lab', 'file_type'],
        excluded_embed_metadata_keys=['block_type', 'file']
    )
    if pd.notna(row['previous_node_id']):
        node.relationships[NodeRelationship.PREVIOUS] = RelatedNodeInfo(
            node_id=row['previous_node_id']
        )
    if pd.notna(row['next_node_id']):
        node.relationships[NodeRelationship.NEXT] = RelatedNodeInfo(
            node_id=row['next_node_id']
        )
    return node

In [9]:
df['documents'] = df.apply(create_document, axis=1)

In [10]:
documents = df['documents'].tolist()
documents[0]

Document(id_='fda69868-2c0a-404d-8a76-97e4b57a8a0b', embedding=None, metadata={'block_type': 'Author Names', 'section_title': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks', 'description': 'Lists the names of authors, potentially with affiliations indicated by symbols.', 'file_type': 'application/pdf', 'languages': "['eng']", 'page_number': 1, 'filename': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf', 'authors': 'Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fabio Petroni,Vladimir Karpukhin,Naman Goyal,Heinrich KÃ¼ttler,Mike Lewis,Wen-tau Yih,Tim RocktÃ¤schel,Sebastian Riedel,Douwe Kiela', 'research_lab': 'Facebook AI Research,University College London,New York University', 'publication_year': 2020, 'title_of_the_document': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'}, excluded_embed_metadata_keys=['block_type', 'file'], excluded_llm_metadata_keys=['authors', 'research_lab', 'file_type'], relationships={<NodeRelationship

In [14]:
from dotenv import load_dotenv
import os

load_dotenv()

credentials_path = os.getenv('GCP_CREDENTIALS_PATH')
if credentials_path is None:
    raise ValueError("La variable de entorno 'GCP_CREDENTIALS_PATH' no está definida.")


In [15]:
embedding = VertexIEmbeddings(credentials_path=credentials_path)
Settings.llm = Vertex(model="gemini-1.5-pro-001")
Settings.embed_model = embedding

semantic_spliter = SemanticSplitterNodeParser(
    buffer_size=1,
    embed_model=embedding,
    include_metadata=True,
    include_prev_next_rel=True
)

qa_extractor = QuestionsAnsweredExtractor(questions=3)

transformation = [
    semantic_spliter,
    qa_extractor,
    embedding
]

pipeline = IngestionPipeline(transformations=transformation)



nodes = pipeline.run(documents=documents, in_place=True, show_progress=True)




Parsing nodes:   0%|          | 0/159 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/9 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

  9%|▉         | 27/287 [00:31<04:27,  1.03s/it]Retrying llama_index.llms.vertex.utils.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
Retrying llama_index.llms.vertex.utils.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-pro. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..
 10%|█         | 29/287 [00:32<03:46,  1.14it/s]Retrying llama_index.llms.vertex.utils.acompletion_with_retry.<locals>._completion_with_retry in 4.0 seconds as 

Generating embeddings:   0%|          | 0/287 [00:00<?, ?it/s]

In [17]:
nodes[0]

TextNode(id_='9841e894-8120-4ef5-ba04-33b1fdb56745', embedding=[-0.025246039032936096, -0.003179964842274785, -0.06594706326723099, -8.014733612071723e-05, 0.0015911523951217532, 0.00019661440455820411, 0.024234145879745483, 0.008799227885901928, 0.05155366659164429, -0.006005400791764259, -0.020428717136383057, -0.025113023817539215, 0.025076409801840782, 0.006820478476583958, -0.0062823425978422165, -0.015581350773572922, 0.02629615180194378, 0.06237902119755745, -0.05881671607494354, -0.012944183312356472, -0.01927935518324375, -0.009847882203757763, 0.019640354439616203, -0.03306274488568306, -0.02076777257025242, -0.07714923471212387, 0.05812016874551773, -0.018395598977804184, 0.012749012559652328, -0.00503784092143178, 0.03264058008790016, 0.06509224325418472, 0.028401130810379982, 0.004888472612947226, 0.005833648145198822, 0.060041554272174835, -0.03575453534722328, 0.03037434071302414, 0.03437187522649765, -0.10131872445344925, -0.048668887466192245, 0.0017969397595152259, -0

In [18]:
# Guardar la lista de objetos a un archivo
with open('embeded_nodes_v2.pkl', 'wb') as outp:  # 'wb' para escribir en modo binario
    pickle.dump(nodes, outp, pickle.HIGHEST_PROTOCOL)

In [18]:
# Leer la lista de objetos desde un archivo
with open('embeded_nodes_v2.pkl', 'rb') as inp:  # 'rb' para leer en modo binario
    loaded_objects_list = pickle.load(inp)
loaded_objects_list[0]

TextNode(id_='8ac390ca-3ea4-4bae-b4a2-16b1de584638', embedding=[-0.040346380323171616, -0.022464128211140633, -0.05197039991617203, 0.0053776707500219345, -0.0019971011206507683, 0.0013902459759265184, 0.017468459904193878, 0.024134838953614235, 0.04877280816435814, -0.010017641820013523, -0.030863329768180847, -0.027494480833411217, 0.023229127749800682, 0.013211933895945549, 0.0018636994063854218, -0.021698765456676483, 0.006273228675127029, 0.053895555436611176, -0.0625259280204773, -0.002528643235564232, 0.0072318012826144695, -0.033708736300468445, 0.006138769444078207, -0.047183841466903687, -0.021933559328317642, -0.04295714572072029, 0.05227678641676903, -0.022620609030127525, -0.004851000849157572, -0.006953277625143528, 0.022087588906288147, 0.0642944946885109, 0.015330696478486061, 0.010571449995040894, -0.004879107233136892, 0.05753064900636673, -0.024754401296377182, 0.027319954708218575, 0.02849961631000042, -0.09810864180326462, -0.053667403757572174, 0.01394648384302854

In [19]:
index = VectorStoreIndex(nodes)

In [20]:
index.storage_context.persist(persist_dir='./data/index/vector_index_prueba_v2')

In [54]:
index.docstore.get_document('d3fc8886-2d86-40e2-be54-f37422e67c1a')

TextNode(id_='d3fc8886-2d86-40e2-be54-f37422e67c1a', embedding=None, metadata={'block_type': 'NarrativeText', 'section_title': 'I Number of instances per dataset', 'description': 'This sentence describes the content of Table 7, indicating it shows the number of data points for training, development, and testing in each dataset used.', 'file_type': 'application/pdf', 'languages': "['eng']", 'page_number': 19, 'filename': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.pdf', 'authors': 'Patrick Lewis,Ethan Perez,Aleksandra Piktus,Fabio Petroni,Vladimir Karpukhin,Naman Goyal,Heinrich KÃ¼ttler,Mike Lewis,Wen-tau Yih,Tim RocktÃ¤schel,Sebastian Riedel,Douwe Kiela', 'research_lab': 'Facebook AI Research,University College London,New York University', 'publication_year': 2020, 'title': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'}, excluded_embed_metadata_keys=['title'], excluded_llm_metadata_keys=['authors', 'research_lab'], relationships={<NodeRelation

In [22]:
retriver = index.as_retriever(similarity_top_k=5)

In [23]:
r = retriver.retrieve('what is rag system?')
for x in map(lambda x: (x.text, x.score), r):
    print(x[0], x[1], sep='\n', end='\n\n')

We also compress the document index using FAISS’s compression tools, reducing the CPU memory requirement to 36GB. Scripts to run experiments with RAG can be found at https://github.com/huggingface/transformers/ blob/master/examples/rag/README.md and an interactive demo of a RAG model can be found at https://huggingface.co/rag/
0.58772494529224

1). The retriever (Dense Passage Retriever [26], henceforth DPR) provides latent documents conditioned on the input, and the seq2seq model (BART [32]) then conditions on these latent documents together with the input to generate the output. We marginalize the latent documents with a top-K approximation, either on a per-output basis (assuming the same document is responsible for all tokens) or a per-token basis (where different documents are responsible for different tokens). Like T5 [51] or BART, RAG can be ﬁne-tuned on any seq2seq task, whereby both the generator and retriever are jointly learned.
0.5668941746042162

To train the retriever and 

In [24]:
llm = Settings.llm
llm.complete("Hello this is a sample text").text

'Please provide me with more context or information about what you would like me to do with the sample text "Hello this is a sample text". \n\nFor example, do you want me to:\n\n* Analyze the sentiment of the text?\n* Translate it into another language?\n* Identify the grammatical structure of the sentence?\n* Use it as a starting point for a story?\n\nOnce I know what you\'d like me to do, I can be more helpful! 😊 \n'

In [26]:
query_engine = index.as_query_engine(verbose=True)

In [27]:
print(query_engine.query('what is rag system?').response)

RAG, or Retrieval-Augmented Generation, is a method that combines a "retriever" and a "seq2seq model" to generate text. The retriever selects relevant documents based on the input, and the seq2seq model uses both the input and these documents to produce the output. 



In [28]:
print(query_engine.query('Que es un sistema rag?').response)

Un sistema RAG combina un modelo de generación de lenguaje pre-entrenado con una memoria no paramétrica, como un índice de documentos, para mejorar sus capacidades de generación de texto. 



In [29]:
print(query_engine.query('Dame titulos de documentos que tengas?').response)

Puedo proporcionarte el título del documento "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks". 



In [30]:
print(query_engine.query(
    'Quienes son los autores de: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.').response)

I'm sorry, but I cannot answer that question based on the context provided. 



In [85]:
print(query_engine.query(
    'Puedes resumir el articulo: Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks.').response)

Este artículo explora modelos de lenguaje que combinan memoria paramétrica y no paramétrica para la generación de lenguaje. Se enfoca en modelos RAG que usan un modelo pre-entrenado seq2seq como memoria paramétrica y un índice vectorial denso de Wikipedia como memoria no paramétrica, a la que se accede con un recuperador neuronal pre-entrenado. 

El artículo compara dos formulaciones RAG: una que se condiciona en los mismos pasajes recuperados a través de toda la secuencia generada y otra que puede usar diferentes pasajes por token. Los modelos RAG se evalúan en una amplia gama de tareas de PNL de uso intensivo de conocimiento, incluyendo tareas de preguntas y respuestas de dominio abierto y tareas de generación de lenguaje. 

Los resultados muestran que los modelos RAG superan a los modelos seq2seq paramétricos y a las arquitecturas de recuperación y extracción específicas de la tarea en las tareas de preguntas y respuestas. Para las tareas de generación de lenguaje, los modelos RAG g

In [31]:
chat_engine = index.as_chat_engine(chat_mode="react", verbose=True)

In [32]:
response = chat_engine.chat("Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?")
print(response)

> Running step 723fc0dd-f8b7-4858-b618-50223c179261. Step input: Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is RAG?'}
Observation: RAG, which stands for Retrieval-Augmented Generation, is a method for enhancing pre-trained language models by incorporating a non-parametric memory component. 

> Running step 6aaaffba-54a9-40a1-9fd7-2c072bf86377. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: RAG, o Generación Aumentada por Recuperación, es un método para mejorar los modelos de lenguaje preentrenados al incorporar un componente de memoria no paramétrico. En términos más sencillos, RAG permite que los modelos de lenguaje accedan y utilicen información externa, como documentos o bases de datos, para generar respuestas más completas y preci

In [33]:
chat_engine.chat('Recuerdas como me llamo?')

> Running step e00b68f1-e710-4ee8-b1f3-23f7d1a15c1a. Step input: Recuerdas como me llamo?
Thought: (Implicit) I can answer without any more tools!
Answer: Sí, te llamas Nicolas. 



AgentChatResponse(response='Sí, te llamas Nicolas. \n', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)

In [34]:
streaming_response = chat_engine.stream_chat("Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?")
for token in streaming_response.response_gen:
    print(token, end="")

> Running step 15cae485-4d67-41f4-8d69-9fe89ed0bd07. Step input: Hola soy Nicolas, me puedes dar informacion sobre el sistema RAG?
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: query_engine_tool
Action Input: {'input': 'What is RAG?'}
Observation: RAG, which stands for Retrieval-Augmented Generation, is a method for enhancing pre-trained language models by incorporating a non-parametric memory component. 

> Running step 4ffb67c7-18ef-4455-ad3c-2b8c5b26f5fa. Step input: None
's language to answer
Answer: RAG, o Generación Aumentada por Recuperación, es un método para mejorar los modelos de lenguaje preentrenados al incorporar un componente de memoria no paramétrico. En términos más sencillos, RAG permite que los modelos de lenguaje accedan y utilicen información externa, como documentos o bases de datos, para generar respuestas más completas y precisas. 
